### 衆議院の小選挙区の当選人数シミュレーション
#### 状況を簡単にするために、以下の仮定を置く
#### 1. 各小選挙区に各政党の候補者が1人ずつ立候補する。
#### 2. 投票者は、あらかじめ指定された確率で、いずれかの政党をランダムに支持するものとする。
#### 3. 投票者は、支持政党からの立候補者に投票するものとする。
#### 4. 各小選挙区の投票数の格差は最大6倍で、その格差は一様分布に従うとする。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 各政党の支持率
supporter_ratio = [30, 29, 28, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
prob = supporter_ratio / np.array(supporter_ratio).sum()
print(prob)

各政党の支持率： [0.21126761 0.20422535 0.1971831  0.07042254 0.06338028 0.05633803
 0.04929577 0.04225352 0.03521127 0.02816901 0.02112676 0.01408451
 0.00704225]


In [3]:
# 人口と投票率の設定
# 人口1.2億。全世代の人口は等分布。平均寿命85歳、投票いけるのは80歳まで。そのうち50%が投票するとする。
population = 12000000 
electorate = population*(80-18)/85 
voting_rate = 0.5
voting_population = int(electorate * voting_rate) 

In [4]:
# 有権者ごとに投票した政党を確率でランダムに与える
N = len(supporter_ratio) # 政党数
dice = list(range(N))
X =  np.random.choice(a=dice, size=voting_population, p=prob)

In [5]:
# 有権者が投票した政党を小選挙区ごとに分割
num_of_small_electoral_district = 289 # 小選挙区の数
disparity = 6 # 最大格差
a = np.random.uniform(1, disparity, num_of_small_electoral_district)
eachsize_of_small_electoral_district = voting_population * a / a.sum()
cumsum = np.cumsum(eachsize_of_small_electoral_district).astype(int)
X_split = np.array_split(X, cumsum[:-1])

# 各小選挙区内ごとに支持者の分布は完全にランダム、小選挙区ごとの投票数は均等とするときは以下。
# X_split = np.array_split(X, num_of_small_electoral_district)

In [6]:
# 小選挙区ごとの結果
result = np.zeros(num_of_small_electoral_district)
for i in range(len(X_split)):
    a = X_split[i]
    u, counts = np.unique(a, return_counts=True)
    result[i]= np.argmax(counts)
# result

In [7]:
# 議席数
num_of_seat = np.zeros(N)
for i in range(N):
    num_of_seat[i] = np.nonzero(result==i)[0].shape[0]
num_of_seat

array([262.,  23.,   4.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.])

#### 支持率21.1%の政党が262議席、20.4%の政党が23議席、19.7%の政党が4議席、他の政党は0議席。
#### これがあってるとすると、小選挙区ってマジキチ、、、